# 정수 인코딩(Integer Encoding)
- 컴퓨터는 텍스트보다 숫자를 더 효율적으로 처리
- 자연어처리에서는 이 task에 대한 여러 방법이 존재
- 그 기법들의 기본은 단어를 정수에 mapping
- 보통 단어의 빈도수를 기반으로 index 번호 지정

## 1. 정수 인코딩(Integer Encoding)
- 단어에 정수를 부여하는 방법 중 하나로 단어를 빈도수 순으로 정렬한 단어 집합(vocabulary) 생성
- 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여


### 1) dictionary 사용

In [1]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

In [2]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."
text

'A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain.'

In [3]:
# 문장 토큰화
text = sent_tokenize(text)
text

['A barber is a person.',
 'a barber is good person.',
 'a barber is huge person.',
 'he Knew A Secret!',
 'The Secret He Kept is huge secret.',
 'Huge secret.',
 'His barber kept his word.',
 'a barber kept his word.',
 'His barber kept his secret.',
 'But keeping and keeping such a huge secret to himself was driving the barber crazy.',
 'the barber went up a huge mountain.']

In [4]:
# 정제와 단어 토큰화
vocab = {}
sentences = []
stop_words = stopwords.words("english")

for sent in text:
    sentence = word_tokenize(sent)
    result = []
    
    for word in sentence:
        word = word.lower() # 모든 단어 소문자화
        if word not in stop_words: # 불용어 제외
            if len(word) > 2: #단어 길이가 3이상만
                result.append(word)
                
                if word not in vocab.keys():
                    vocab[word] = 0
                vocab[word] += 1
    sentences.append(result)
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [5]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [6]:
#빈도수 기준으로 vocab 정렬
import operator
vocab_sorted = sorted(vocab.items(), key=operator.itemgetter(1), reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [7]:
#w2i 구현
w2i = {}
i = 0
for word,freq in vocab_sorted:
    if freq > 1: #빈도수가 2회 이상이면 추가
        i += 1
        w2i[word] = i
print(w2i)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


이 중 빈도수가 높은 순으로 n 개의 vocab_size만 사용하고자 할경우!

In [10]:
# vocab_size 지정 및 정리
vocab_size = 5
words_frequency = [w for w,c in w2i.items() if c> vocab_size]

for w in words_frequency:
    del w2i[w]
print(w2i)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [11]:
#OOV 토큰 추가
w2i["OOV"] = len(w2i)+1
w2i

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}

In [12]:
# 정수 인코딩
encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(w2i[w])
        except:
            temp.append(w2i["OOV"])
    encoded.append(temp)
print(sentences)
print(encoded)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


### 2) Counter 사용

In [18]:
from collections import Counter
from itertools import chain

In [19]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [24]:
# 단어별 빈도수 카운트
vocab = Counter(chain(*sentences))
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [26]:
#vocab_size=5로 지정!
vocab = vocab.most_common(5)
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [28]:
# w2i 생성
w2i = {word : i for i, (word, _) in enumerate(vocab,start=1)}
print(w2i)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
